<a href="https://colab.research.google.com/github/ChaejinE/HanComTermProject/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, Model, losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f'x_train {x_train.shape}')
print(f'x_test {x_test.shape}')
print(f'y_train {y_train.shape}')
print(f'y_test {y_test.shape}')

x_train (60000, 28, 28)
x_test (10000, 28, 28)
y_train (60000,)
y_test (10000,)


In [6]:
x_train = tf.pad(x_train, [[0, 0], [2, 2], [2, 2]])/255
x_test = tf.pad(x_test, [[0, 0], [2, 2], [2, 2]])/255
print(f'pad x_train {x_train.shape}')
print(f'pad x_test {x_test.shape}')

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
print(f'expand x_train {x_train.shape}')
print(f'expand x_test {x_test.shape}')

x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
print(f'repeat x_train {x_train.shape}')
print(f'repeat x_test {x_test.shape}')

pad x_train (60000, 32, 32)
pad x_test (10000, 32, 32)
expand x_train (60000, 32, 32, 1)
expand x_test (10000, 32, 32, 1)
repeat x_train (60000, 32, 32, 3)
repeat x_test (10000, 32, 32, 3)


In [7]:
%tensorflow_version 2.x
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.5.0
/device:GPU:0
Num GPUs Available:  1


In [8]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [9]:
def inception(x,
              filters_1x1,
              filters_3x3_reduce,
              filters_3x3,
              filters_5x5_reduce,
              filters_5x5,
              filters_pool):
  path1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)

  path2 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
  path2 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(path2)

  path3 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
  path3 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(path3)

  path4 = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
  path4 = Conv2D(filters_pool, (1,1), padding='same', activation='relu')(path4)

  return tf.concat([path1, path2, path3, path4], axis=3)

In [12]:
num_classes = 10
img_height = 224
img_width = 224

with tf.device('/device:GPU:0'):

  input = layers.Input(shape=(32, 32, 3))
  input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation='bilinear', input_shape=x_train.shape[1:])(input)

  x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input_tensor)
  x = layers.MaxPooling2D(3, strides=2)(x)

  x = layers.Conv2D(64, 1, strides=1, padding='same', activation='relu')(x)
  x = layers.Conv2D(192, 3, strides=1, padding='same', activation='relu')(x)

  x = layers.MaxPooling2D(3, strides=2)(x)

  x = inception(x,
              filters_1x1=64,
              filters_3x3_reduce=96,
              filters_3x3=128,
              filters_5x5_reduce=16,
              filters_5x5=32,
              filters_pool=32)
  
  x = inception(x,
              filters_1x1=128,
              filters_3x3_reduce=128,
              filters_3x3=192,
              filters_5x5_reduce=32,
              filters_5x5=96,
              filters_pool=64)
  
  x = layers.MaxPooling2D(3, strides=2)(x)

  x = inception(x,
            filters_1x1=192,
            filters_3x3_reduce=96,
            filters_3x3=208,
            filters_5x5_reduce=16,
            filters_5x5=48,
            filters_pool=64)
  
  aux1 = layers.AveragePooling2D((5, 5), strides=3)(x)
  aux1 = layers.Conv2D(128, 1, padding='same', activation='relu')(aux1)
  aux1 = layers.Flatten()(aux1)
  aux1 = layers.Dense(1024, activation='relu')(aux1)
  aux1 = layers.Dropout(0.7)(aux1)
  aux1 = layers.Dense(num_classes, activation='softmax')(aux1)

  x = inception(x,
            filters_1x1=160,
            filters_3x3_reduce=112,
            filters_3x3=224,
            filters_5x5_reduce=24,
            filters_5x5=64,
            filters_pool=64)

  x = inception(x,
            filters_1x1=128,
            filters_3x3_reduce=128,
            filters_3x3=256,
            filters_5x5_reduce=24,
            filters_5x5=64,
            filters_pool=64)

  x = inception(x,
            filters_1x1=112,
            filters_3x3_reduce=144,
            filters_3x3=288,
            filters_5x5_reduce=32,
            filters_5x5=64,
            filters_pool=64)
  
  aux2 = layers.AveragePooling2D((5, 5), strides=3)(x)
  aux2 = layers.Conv2D(128, 1, padding='same', activation='relu')(aux2)
  aux2 = layers.Flatten()(aux2)
  aux2 = layers.Dense(1024, activation='relu')(aux2)
  aux2 = layers.Dropout(0.7)(aux2)
  aux2 = layers.Dense(num_classes, activation='softmax')(aux2)

  x = inception(x,
            filters_1x1=256,
            filters_3x3_reduce=160,
            filters_3x3=320,
            filters_5x5_reduce=32,
            filters_5x5=128,
            filters_pool=128)

  x = layers.MaxPooling2D(3, strides=2)(x)

  x = inception(x,
          filters_1x1=256,
          filters_3x3_reduce=160,
          filters_3x3=320,
          filters_5x5_reduce=12,
          filters_5x5=128,
          filters_pool=128)

  x = inception(x,
            filters_1x1=384,
            filters_3x3_reduce=192,
            filters_3x3=384,
            filters_5x5_reduce=48,
            filters_5x5=128,
            filters_pool=128) 
  
  x = layers.GlobalAveragePooling2D()(x)

  x = layers.Dropout(0.4)(x)
  out = layers.Dense(num_classes, activation='softmax')(x)

  google_model = Model(inputs = input, outputs = [out, aux1, aux2])
  opt = Adam(learning_rate=0.001)
  google_model.compile(optimizer='adam',
                       loss=[losses.sparse_categorical_crossentropy, losses.sparse_categorical_crossentropy, losses.sparse_categorical_crossentropy],
                       loss_weights=[1, 0.3, 0.3],
                       metrics=['accuracy'])
  
  

In [13]:
google_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
resizing_2 (Resizing)           (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_118 (Conv2D)             (None, 112, 112, 64) 9472        resizing_2[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_26 (MaxPooling2D) (None, 55, 55, 64)   0           conv2d_118[0][0]                 
____________________________________________________________________________________________